# Model bez cech globalnych w wewnętrznej warstwie

In [1]:
import torch

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset_big.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset_big.pt')

print(len(train_dataset))
print(len(test_dataset))

1840
460


# Definicja modelu

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Inicjalizacja modelu bez cech globalnych
Wykorzystane zostaną parametry na podstawie naszych wcześniejszych badań, zatem zakładam:
* hidden_dim = 192
* dropout = 0.1

In [15]:
import torch
from torch_geometric.loader import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

hidden_dim = 192
dropout = 0.1

model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout)

train_loader = DataLoader(train_dataset, batch_size=1 , shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

cpu
Epoch: 0


C:\Users\jderd\AppData\Local\Temp\ipykernel_18776\1814573080.py:32: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)


Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40


In [26]:
from sklearn.metrics import r2_score, mean_squared_error

y_true = []
y_pred = []

model.eval()

for data in test_loader:
    with torch.no_grad():
        prediction = model(data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(data.y.item())

r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

R2 Score = -0.0011
MSE Score = 52549.9436
RMSE Score = 229.2377
